In [ ]:
%matplotlib inline

from matplotlib import rc
from P3.aoSystem.fourierModel import *
from P3.aoSystem.FourierUtils import *
from mastsel import *
rc("text", usetex=False)

In [ ]:
path = "../data/ini/"
parametersFile = 'mavisParamsTests'
fullPathFilename = path + parametersFile + '.ini'
parser = ConfigParser()
parser.read(fullPathFilename);
wvl = eval(parser.get('sources_science', 'Wavelength'))[0]  # lambda
pixel_psf = eval(parser.get('telescope', 'TelescopeDiameter'))      # mas

In [ ]:
N = 960
fao = fourierModel(fullPathFilename, calcPSF=False, verbose=True, display=False)
freq_range = fao.ao.cam.fovInPix*fao.freq.PSDstep # fao.psf_FoV/fao.wvlRef/206264.8
inputPsf = Field(wvl, N, freq_range, 'rad')
inputPsf.loadSamplingFromFile('../data/convTestData/psf_for_convol.fits')
convolutionKernel = Field(wvl, N, freq_range, 'rad')
convolutionKernel.loadSamplingFromFile('../data/convTestData/fourierkernel_for_convol.fits')
result = Field(wvl, N, freq_range, 'rad')
result.loadSamplingFromFile('../data/convTestData/final_psf_convol.fits')
myresult = convolve(inputPsf, convolutionKernel)
result.standardPlot(True)
myresult.standardPlot(True)
samplingsifference = cp.asnumpy(myresult.sampling) - cp.asnumpy(result.sampling)
print(np.max(samplingsifference), np.min(samplingsifference) )

In [ ]:
convolutionKernel.standardPlot(False)
myConvKernel = residualToSpectrum([np.pi-1.0472, 4.2661, 2.12330], wvl, N, 1.163545*960 ) # fao.psf_FoV*fao.psInMas
myConvKernel.standardPlot(False)

dd = Field(wvl, N, freq_range, 'rad')

dd.sampling = convolutionKernel.sampling - myConvKernel.sampling
dd.standardPlot(False)
